# Problemas de Multicolinealidad


¿Qué ocurre cuándo la matriz de diseño **NO** es de rango completo?

$$\text{Rank}(M) < P \Rightarrow \det( X X^T ) = 0 \Rightarrow \not\exists \ ( X^T X )^{-1}$$

Y la matriz de diseño no es de rango completo si alguna covariable es combinación lineal de otra(s).

En la práctica suele la colinealidad exacta no ocurre con frecuencia, sin embargo, la colinealidad *aproximada* si lo es.

Es decir, una variable se puede aproximar como una combinación lineal de otra covariable(s). O en términos matemáticos:

$$\det( X X^T ) \approx 0$$

Hay algunas maneras de "saber" si existe colinealidad en la matriz de diseño.

**METODO 1** Calcular $\det( X X^T )$

Si el determinante es cercano a cero, tenemos indicios de multicolinealidad.
En caso contrario, podemos decir que no hay multicolinealidad.

**METODO 2** Calcular la matriz de correlación (lineal).

Bajo el supuesto de que no hay multicolinealidad, entonces la matriz de correlación debe ser aproximadamente la matriz identidad, y el determinante de la matriz identidad es uno.
Si el determinante de la matriz de correlación es cercano a uno, podemos decir que no hay multicolinealidad.
caso contrario, cuando hay multicolinealidad el determinante de la matriz de correlación se acercará a cero.

**METODO 3** Calcular el Indice de Condición de la Matriz (Valores propios de la matriz de correlación)

Se tiene que $\left| X^T X \right| = \prod_{i=1}^n \lambda_j$ donde $\lambda_j$ son los $j$-ésimos autovalores de la matriz $ X^T X$.
La cantidad de autovalores cercanos a cero indica la cantidad de covariables que pueden ser expresadas como combinación lineal de otras.
Para facilitar el análisis y simplemente indicar si hay o no multicolinealidad, se calcula el Indice de Condición de la matriz.
$$IC = \sqrt{ \frac{ \lambda_{max} }{ \lambda_{min} } }$$
Se suele considerar que hay multicolinealidad fuerte si $IC > 30$, y hay multicolinealidad moderada si $10 \leq IC \leq 30$.

**METODO 4** Calcular el Factor de Inflación de Varianza (VIF)

Este método es uno de los más usados porque indica qué variable puede ser expresada como combinación lineal de otra.
Por lo tanto, existe un *VIF* para cada covariable. La idea es hacer una regresión lineal para cada covariable tomandose a ella mismo como la variable explicativa y a las demás como las variables predictores (covariables), posteriormente calcular su $R^2$ y realizar el siguiente cálculo
$$VIF_i = \frac{ 1 }{ 1 - R^2_i } $$

Algunos autores comentan que dicha variable puede ser expresada como combinación lineal de las demás si $VIF_i$ toma valores superiores a 50 (lo que equivale a un $R^2 = 0.98$)
Sin embargo, existe una prueba no paramétrica para el *VIF*, donde podemos decir que existe colinealidad para la variable $i$ si 
$$VIF_i > \frac{ p-2 }{ n - p + 1 } F_0 + 1 $$
donde $F_0$ es el cuantil de la distribución F de Fisher-Snedecor con p-2 y n-p+1 grados de libertad, $n$ es el tamaño de muestra y $p$ es el número de covariables en la matriz de diseño (incluyendo intercepto).

# Ejemplo Práctico

In [1]:
# Bibliotecas a utilizar 
import numpy                   as np      # Uso de álgebra lineal, funciones vectoriales
import pandas                  as pd      # Trabajar con DataFrames
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF

In [2]:
# Leemos los datos MTCARS y Creamos nuestra matriz de diseño
# Los datos de este conjunto de datos se usan para estimar las millas por galon (mpg) ( problema de regresión )
DATOS = pd.read_csv("mtcars.csv")
M     = np.matrix( DATOS.values[:,1:11] )
DATOS.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [3]:
# Realizamos el calculo del determinante, la matriz de correlaciones, el VIF para cada una de las variables y los autovalores
determinante_m    = np.linalg.det( a = M.T * M )
mat_correlaciones = np.corrcoef( x = M, rowvar = False )
determinante_cor  = np.linalg.det( a = mat_correlaciones )
VIF               = pd.Series([ variance_inflation_factor( exog = M, exog_idx = k) for k in range(10) ], index = DATOS.columns[1:11] ) 
autovalores, autovectores = np.linalg.eig( M.T * M )
indice_condicion  = np.sqrt( np.max( autovalores ) / np.min( autovalores ) )

print( "Determinante de M.T * M" )
print( determinante_m )
print("")
print( "Determinante de Matriz de Correlaciones" )
print( determinante_cor )
print("")
print( "VIF")
print( VIF )
print("")
print( "R2 de cada VIF")
print( 100*(VIF - 1)/VIF  )
print("")
print( "Autovalores" )
print( autovalores )
print("")
print( "IC" )
print( indice_condicion )


Determinante de M.T * M
3.715650728296922e+19

Determinante de Matriz de Correlaciones
1.5707031674868462e-06

VIF
cyl     112.629828
disp     98.930791
hp       56.047781
drat    132.214353
wt      182.948049
qsec    317.534376
vs        8.752581
am        7.412020
gear    119.804879
carb     32.213836
dtype: float64

R2 de cada VIF
cyl     99.112136
disp    98.989192
hp      98.215808
drat    99.243652
wt      99.453397
qsec    99.685073
vs      88.574799
am      86.508400
gear    99.165309
carb    96.895744
dtype: float64

Autovalores
[2.97332782e+06 5.11326259e+04 2.58522370e+03 3.19264656e+01
 1.83100575e+01 1.63527296e+01 2.63822516e+00 1.97384077e+00
 1.24046003e+00 1.53094049e+00]

IC
1548.210501429205
